In [1]:
import warnings
warnings.filterwarnings('ignore')

%load_ext rpy2.ipython

%run /gfs/devel/ddanko/notebook-init.py

# load R libraries                                                                                    
%R invisible(library(ggplot2))
%R invisible(library(fastcluster))
%R invisible(library(reshape))
%R invisible(library(reshape2))
%R invisible(library(gplots))
%R invisible(library(RSQLite))

#set up ggplot2 defaults                                                                              
%R theme_set(theme_gray(base_size=18))

%pylab inline

pylab.rcParams['figure.figsize'] = (10, 6)


froot = '/gfs/work/ddanko/taxonomic_profiler_comparison/analysis/actual/'

Populating the interactive namespace from numpy and matplotlib


In [ ]:
saved = {
    
    
}

In [32]:
import os
import glob
import subprocess
    
def countSeqs(seqfile):
    cmd = "zcat {} | wc -l".format(seqfile)
    out = subprocess.check_output(cmd,shell=True)
    return int(out) / 4

def findTimingFiles(seqfile):
    root = os.path.dirname(seqfile)
    base = os.path.basename(seqfile)
    base = base[:base.index('.fastq.gz')]
    
    adFile = '{}/acdmnd.dir/{}.fa.gz.acdmnd.timing'.format(root,base)
    adfFile = '{}/acdmndfast.dir/{}.fa.gz.acdmndfast.timing'.format(root,base)
    kFile = '{}/kraken.dir/{}.kraken.timing'.format(root,base)
    dFile = '{}/dmnd.dir/{}.fa.gz.dmnd.timing'.format(root,base)
    dfFile = '{}/dmndfast.dir/{}.fa.gz.dmndfast.timing'.format(root,base)
    
    files = {}
    if os.path.isfile(adFile):
        files['acdmnd'] = adFile
    if os.path.isfile(adfFile):
        files['acdmndfast'] = adfFile
    if os.path.isfile(dFile):
        files['dmnd'] = dFile
    if os.path.isfile(dfFile):
        files['dmndfast'] = dfFile
    if os.path.isfile(kFile):
        files['kraken'] = kFile
        
    return files

def processTimingFiles(seqfile,timingfiles,saved={}):
    out = {}
#     if seqfile in saved:
#         nseqs = saved[seqfile]
#     else:
#         nseqs = countSeqs(seqfile)
    for tool, timingfile in timingfiles.items():
        out[tool] = 0
        with open(timingfile) as tf:
            for line in tf:
                if ':' in line:
                    line = line.split(':')
                    field = line[0]
                    content = line[1]
                    if 'user time' in field.lower():
                        out[tool]+= float(content.strip())
                    elif 'system time' in field.lower():
                        out[tool] += float(content.strip())

    return out
        

def findSet(fastqglob,saved={}):
    fastqs = glob.glob(fastqglob)
    allfiles = {}
    for fastq in fastqs:
        base = os.path.basename(fastq)
        base = base[:base.index('.fastq.gz')]
        timingfiles = findTimingFiles(fastq)
        counts = processTimingFiles(fastq,timingfiles,saved=saved)
        allfiles[base] = counts
    return allfiles




In [33]:
fastqglob = '/gfs/work/ddanko/taxonomic_profiler_comparison/analysis/actual/*/*.fastq.gz'
allfiles = findSet(fastqglob)


In [34]:
# totaltimes = {}
# for sample, tools in allfiles.items():
#     totaltimes[sample] = {}
#     for tool, times in tools.items():
#         totaltime = 0
#         for time in times.values():
#             totaltime += time
#         totaltimes[sample][tool] = totaltime 


import numpy as np
import pandas as pd

df = pd.DataFrame(allfiles)
df = df.transpose()
df.div(3600)


,acdmnd,acdmndfast,dmnd,dmndfast,kraken
cowrumen-dna-R983262,NaN,0.000000,0.0,0.0,4.857906
cowrumen-dna-R983263,NaN,0.000000,0.0,0.0,5.475286
cowrumen-dna-R983264,NaN,0.000000,NaN,0.0,5.767867
cowrumen-dna-R983265,NaN,0.000000,NaN,0.0,4.926017
cowrumen-dna-R983266,NaN,0.000000,NaN,NaN,4.843011
cowrumen-dna-R983267,NaN,0.000000,NaN,NaN,6.432317
cowrumen-dna-R983268,NaN,0.000000,NaN,NaN,6.246667
cowrumen-dna-R983269,NaN,0.089025,NaN,NaN,4.479828
filtered.stool-day0-R1,540.378219,83.168872,NaN,NaN,NaN
filtered.stool-day0-R2,510.191136,78.125889,NaN,NaN,NaN
